In [40]:
import sys, os, collections
from pathlib import Path
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
%matplotlib inline
#lets examine our data
#run locally fname = r'D:\NayaMLCourse\Regressionexcerices\data.csv'
fname = r'data.csv'
df = pd.read_csv(fname, index_col=0)
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

In [29]:
#letes see the correlation ofthe numeric fields
plt.figure(figsize=(12,10))
df_num=df.select_dtypes(include = ['float64', 'int64'])
df_num= df_num.groupby(df_num.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))
corr = df_num.iloc[:, 1:].corr().round(2)
corr.style.background_gradient(cmap="coolwarm")


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
LotFrontage,1,0.31,0.23,-0.05,0.12,0.08,0.18,0.22,0.04,0.12,0.36,0.41,0.07,0.04,0.37,0.09,-0.01,0.18,0.05,0.24,-0.01,0.32,0.24,0.06,0.27,0.32,0.08,0.14,0.01,0.06,0.04,0.18,0,0.01,0.01,0.33
LotArea,0.31,1,0.11,-0.01,0.01,0.01,0.1,0.21,0.11,-0,0.26,0.3,0.05,0,0.26,0.16,0.05,0.13,0.01,0.12,-0.02,0.19,0.27,-0.02,0.15,0.18,0.17,0.08,-0.02,0.02,0.04,0.08,0.04,0,-0.01,0.26
OverallQual,0.23,0.11,1,-0.09,0.57,0.55,0.41,0.24,-0.06,0.31,0.54,0.48,0.3,-0.03,0.59,0.11,-0.04,0.55,0.27,0.1,-0.18,0.43,0.4,0.52,0.6,0.56,0.24,0.31,-0.11,0.03,0.06,0.07,-0.03,0.07,-0.03,0.79
OverallCond,-0.05,-0.01,-0.09,1,-0.38,0.07,-0.13,-0.05,0.04,-0.14,-0.17,-0.14,0.03,0.03,-0.08,-0.05,0.12,-0.19,-0.06,0.01,-0.09,-0.06,-0.02,-0.31,-0.19,-0.15,-0,-0.03,0.07,0.03,0.05,-0,0.07,-0,0.04,-0.08
YearBuilt,0.12,0.01,0.57,-0.38,1,0.59,0.31,0.25,-0.05,0.15,0.39,0.28,0.01,-0.18,0.2,0.19,-0.04,0.47,0.24,-0.07,-0.17,0.1,0.15,0.78,0.54,0.48,0.22,0.19,-0.39,0.03,-0.05,0,-0.03,0.01,-0.01,0.52
YearRemodAdd,0.08,0.01,0.55,0.07,0.59,1,0.18,0.13,-0.07,0.18,0.29,0.24,0.14,-0.06,0.29,0.12,-0.01,0.44,0.18,-0.04,-0.15,0.19,0.11,0.62,0.42,0.37,0.21,0.23,-0.19,0.05,-0.04,0.01,-0.01,0.02,0.04,0.51
MasVnrArea,0.18,0.1,0.41,-0.13,0.31,0.18,1,0.26,-0.07,0.11,0.36,0.34,0.17,-0.07,0.39,0.09,0.03,0.28,0.2,0.1,-0.04,0.28,0.25,0.25,0.36,0.37,0.16,0.12,-0.11,0.02,0.06,0.01,-0.03,-0.01,-0.01,0.48
BsmtFinSF1,0.22,0.21,0.24,-0.05,0.25,0.13,0.26,1,-0.05,-0.5,0.52,0.45,-0.14,-0.06,0.21,0.65,0.07,0.06,0,-0.11,-0.08,0.04,0.26,0.15,0.22,0.3,0.2,0.11,-0.1,0.03,0.06,0.14,0,-0.02,0.01,0.39
BsmtFinSF2,0.04,0.11,-0.06,0.04,-0.05,-0.07,-0.07,-0.05,1,-0.21,0.1,0.1,-0.1,0.01,-0.01,0.16,0.07,-0.08,-0.03,-0.02,-0.04,-0.04,0.05,-0.09,-0.04,-0.02,0.07,0,0.04,-0.03,0.09,0.04,0,-0.02,0.03,-0.01
BsmtUnfSF,0.12,-0,0.31,-0.14,0.15,0.18,0.11,-0.5,-0.21,1,0.42,0.32,0,0.03,0.24,-0.42,-0.1,0.29,-0.04,0.17,0.03,0.25,0.05,0.19,0.21,0.18,-0.01,0.13,-0,0.02,-0.01,-0.04,-0.02,0.03,-0.04,0.21


<Figure size 864x720 with 0 Axes>

In [ ]:
#we can plot all of them and loom it in a visual way:
#numeric_cpoumns=df.select_dtypes(include = ['float64', 'int64'])
#sns.pairplot(numeric_cpoumns)

In [20]:
#now lets examine the most correlated featutes that we saw in the corr table and see the error of it
def rmse(y_true, y_pred):
    return (((y_true-y_pred)**2).mean())**0.5

In [21]:
#lets define x and y and then linear model
from sklearn.linear_model import LinearRegression
Features=["OverallQual","BsmtFinSF1"]
model = LinearRegression()
X=df_num[Features]
y=np.log(df["SalePrice"])
model.fit(X,y)
y_predict=model.predict(X)
#print (model.intercept_)
#print (model.coef_)
err = rmse(y,y_predict)
print("both features error:",err)

both features error: 0.2184518870254415


In [35]:
#the lowest error that we got with numeric is comming from the following features:
#lets define x and y and then linear model, thisis the lower i could get , err=37247.412828165805
from sklearn.linear_model import LinearRegression
Features=["OverallQual","BsmtFinSF1","LotArea","GrLivArea","BedroomAbvGr","YearBuilt", "MasVnrArea","GarageYrBlt","GarageCars"] 
model = LinearRegression(fit_intercept=False)
X=df_num[Features]
y=np.log(df["SalePrice"])
model.fit(X,y)
y_predict=model.predict(X)
#print (model.intercept_)
#print (model.coef_)
err = rmse(y,y_predict)
print("all features error:",err)

all features error: 0.17950588808375914


In [42]:
#lest start looking on the categorial features
#categorial
df_object = df.select_dtypes(include = ['object'])
df_object.head()
df_cat = pd.get_dummies(df_object ,columns=df_object.columns)
#now lets add the price as weel to this data frame 
df_cat['price']=df["SalePrice"]
corr = df_cat.iloc[:, 1:].corr().round(2)

In [24]:
#now lets look on the features with the higer correlation values to the price(this is what interesting for us)
#for example will start with  thresholdof 0.4 (abs,+-)
TH=0.4
cor_target=abs(corr["price"])
relevant_features=cor_target[cor_target>TH]
relevant_features

KeyError: 'price'

In [43]:
#now we will map the relevant to our main data frame
df["ExterQual_num"]=df["ExterQual"].map({k:i+1 for i,k in enumerate(['Ex', 'Gd', 'TA', 'Fa', 'Po'])})
df["BsmtQual_num"]=df["BsmtQual"].map({k:i+1 for i,k in enumerate(['Ex', 'Gd', 'TA', 'Fa', 'Po','NA'])})
df["KitchenQual_num"]=df["KitchenQual"].map({k:i+1 for i,k in enumerate(['Ex', 'Gd', 'TA', 'Fa', 'Po'])})
df["GarageFinish_num"]=df["GarageFinish"].map({k:i+1 for i,k in enumerate(['Fin', 'RFn', 'Unf', 'NA'])})
df= df.groupby(df.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,ExterQual_num,BsmtQual_num,KitchenQual_num,GarageFinish_num
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,2,2008,WD,Normal,208500,2,2.0,2,2.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,5,2007,WD,Normal,181500,3,2.0,3,2.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,9,2008,WD,Normal,223500,2,2.0,2,2.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,2,2006,WD,Abnorml,140000,3,3.0,2,3.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,12,2008,WD,Normal,250000,2,2.0,2,2.0


In [44]:
#we willdot linear regression tothe features that we suspect
#now we will divide into test and train
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
Features=["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF","KitchenQual_num","FullBath","Fireplaces","YearBuilt","YearRemodAdd","LotArea","BsmtUnfSF","2ndFlrSF","OverallCond","PoolArea"]
model = LinearRegression()

X=df[Features]
y=np.log(df["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1948)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_predict=model.predict(X)
err = rmse(y_train,y_train_pred)
print("all features num and cat error for train:",err)  #only fortest

y_test_pred = model.predict(X_test)
err = rmse(y_test,y_test_pred)
print("all features num and cat error for test:",err)  #we will use the errorvalidating fortest

all features num and cat error for train: 0.15123541437181415
all features num and cat error for test: 0.1525433538920551


In [46]:
#now we willexamine different model
#Lasso
#now lets see other model(regressionand decisiontree)
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

lasso=Lasso(precompute=True,positive=True)
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = 5)
#Features=["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF","KitchenQual_num","FullBath","Fireplaces","YearBuilt","YearRemodAdd","LotArea","BsmtUnfSF","2ndFlrSF","OverallCond","PoolArea"]
Features=["OverallQual","KitchenQual_num","OverallCond"]
X=df[Features]
y=np.log(df["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1948)
lasso_regressor.fit(X_train, y_train)
print("best params:",lasso_regressor.best_params_)
print("best score:",lasso_regressor.best_score_)

best params: {'alpha': 1e-15}
best score: -0.053804948630968255


In [51]:
#Ridge
from sklearn.linear_model import Ridge
import numpy as np
Features=["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF","KitchenQual_num","FullBath","Fireplaces","YearBuilt","YearRemodAdd","LotArea","BsmtUnfSF","2ndFlrSF","OverallCond","PoolArea"]
#Features=["OverallQual","KitchenQual_num","OverallCond"]
X=df[Features]
y=np.log(df["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1948)

clf = Ridge(alpha=0.1)#if wewill do with alpha is around 1 we will have similer number
clf.fit(X_train,y_train) 
Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)
pred_y = clf.predict(X_test)
mae1 = rmse(y_test,pred_y)
print("MAE Test: ", mae1)
pred_y_train  = clf.predict(X_train)
#mae2 = mean_absolute_error(y_train,pred_y)
mae2 = rmse(y_train,pred_y_train)
print("MAE train: ",  mae2)

MAE Test:  0.15253866468531424
MAE train:  0.1512354173507853


In [52]:
#elastic
#elastic net
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
Features=["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF","KitchenQual_num","FullBath","Fireplaces","YearBuilt","YearRemodAdd","LotArea","BsmtUnfSF","2ndFlrSF","OverallCond","PoolArea"]
#Features=["OverallQual","KitchenQual_num","OverallCond"]
X=df[Features]
y=np.log(df["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1948)
regr = ElasticNet(random_state=0)
regr.fit(X_train,y_train)  
#ElasticNet(alpha=0.1, copy_X=True, fit_intercept=False, l1_ratio=0.5,
 #     max_iter=1000, normalize=True, positive=False, precompute=False,
  #    random_state=0, selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=0.1)
#print(regr.coef_) 
#print(regr.intercept_) 
pred_y_test=regr.predict(X_test)
#print(regr.predict(X_test))
mae1 = rmse(y_test,pred_y)
print("MAE Test: ", mae1)
pred_y_train=regr.predict(X_train)
#print(regr.predict(X_test))
mae2 = rmse(y_train,pred_y_train)
print("MAE Train: ", mae2)

MAE Test:  0.15253866468531424
MAE Train:  0.18960538095637758


In [60]:
#now lets see random forests
#Train the model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=500)
Features=["OverallQual","GrLivArea","GarageCars","TotalBsmtSF","KitchenQual_num","Fireplaces","YearBuilt","YearRemodAdd","LotArea","BsmtUnfSF","OverallCond","PoolArea"]
#Features=["OverallQual","GrLivArea","TotalBsmtSF","KitchenQual_num","FullBath","Fireplaces","YearBuilt"]
#Features=["OverallQual","OverallCond"]
y=np.log(df["SalePrice"])
X=df[Features]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1948)
#Fit
model.fit(X_train, y_train)
RandomForestRegressor(bootstrap=True, criterion='rmse', max_depth=5,
           max_features='auto', max_leaf_nodes=50,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=20,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=True)
# mae = mean_absolute_error(y_test,pred_y)
model.fit(X_train,y_train)
pred_y = model.predict(X_test)
#mae1 = mean_absolute_error(y_test,pred_y)
mae1 = rmse(y_test,pred_y)
print("MAE Test: ", mae1)
pred_y = model.predict(X_train)
#mae2 = mean_absolute_error(y_train,pred_y)
mae2 = rmse(y_train,pred_y)
print("MAE train: ",  mae2)  #the ytrain here gotthe better value posibble

MAE Test:  0.14542790024909055
MAE train:  0.05550632924421885


In [59]:
#if we will use the error calculation as:andrun the same random forrest
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=500)
Features=["OverallQual","GrLivArea","GarageCars","TotalBsmtSF","KitchenQual_num","Fireplaces","YearBuilt","YearRemodAdd","LotArea","BsmtUnfSF","OverallCond","PoolArea"]
#Features=["OverallQual","GrLivArea","TotalBsmtSF","KitchenQual_num","FullBath","Fireplaces","YearBuilt"]
#Features=["OverallQual","OverallCond"]
y=np.log(df["SalePrice"])
X=df[Features]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1948)
#Fit
model.fit(X_train, y_train)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=50,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=20,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=True)
# mae = mean_absolute_error(y_test,pred_y)
model.fit(X_train,y_train)
pred_y = model.predict(X_test)
mae1 = mean_absolute_error(y_test,pred_y)
#mae1 = rmse(y_test,pred_y)
print("MAE Test: ", mae1)
pred_y_train = model.predict(X_train)
mae2 = mean_absolute_error(y_train,pred_y_train)
#mae2 = rmse(y_train,pred_y)
print("MAE train: ",  mae2)  #the values are even lower

MAE Test:  0.09909758316152173
MAE train:  0.03633538344882359
